In [1]:
import cv2
import os
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import get_angles   
import csv

pose_model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
key_frames_angles = []

2024-08-09 22:50:43.538940: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 22:50:43.573479: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 22:50:43.798024: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 22:50:44.002359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 22:50:44.102561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
excersise_name = "Bicep_Curls"

In [3]:
csv_file = os.path.join(excersise_name,"key_frame_angles.csv")

with open(csv_file, "r") as f:
    reader = csv.DictReader(f)
    data = [dict(row) for row in reader]

In [5]:
key_frames_angles = [[float(d[key]) for key in d] for d in data]

In [27]:
import numpy as np

mapping_frames = []

def l1_norm(l1, l2):
    if len(l1) != len(l2):
        raise ValueError("Lists must be of the same length.")
    return np.sum(np.abs(np.array(l1) - np.array(l2)))

def get_angles_from_model(model, frame):
    # Convert the frame from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Convert the frame to a TensorFlow tensor
    image = tf.convert_to_tensor(frame_rgb, dtype=tf.uint8)
    
    # Add batch dimension and resize image
    image = tf.expand_dims(image, axis=0)
    image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)
    
    # Load the Movenet model
    movenet = model.signatures['serving_default']
    
    # Get model outputs
    outputs = movenet(image)
    
    # Extract keypoints and calculate angles
    keypoints = outputs['output_0'].numpy().reshape(-1, 3)
    angles = get_angles.get_angles(keypoints)
    
    return angles, keypoints
    
def process_frame(frame, frame_id_from_trainer):
    angles1,keypoints = get_angles_from_model(pose_model,frame)
    angles_only = [float(angles1[key]) for key in angles1]
    closest_frame = None
    match  = -1
    if(frame_id_from_trainer == len(key_frames_angles)):
        return 0
    d = l1_norm(key_frames_angles[frame_id_from_trainer], angles_only)
    if d<:
        match = frame_id_from_trainer + 1
    mapping_frames.append({'keyframe':key_frames_angles[match],'userframe':angles_only,'excercise_phase':'dummy'})
    ## Show Key points
    return match
    

In [28]:
def main():
    video = cv2.VideoCapture("video.mp4")
    count = 0
    wait_count = 0
    frame_id = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        
        # Skip 4 out of 5 frames
        process_frame(frame, frame_id)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        count+=1
    video.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [32]:
X = np.array([d['userframe'] for d in mapping_frames])
Y = np.array([d['keyframe'] for d in mapping_frames])

import numpy as np
from scipy.optimize import least_squares

def affine_transformation(X, Y):
    n, m = X.shape

    def residuals(params):
        A = params[:m*m].reshape(m, m)
        b = params[m*m:]
        return (Y - (X @ A + b)).flatten()

    initial_guess = np.zeros((m * m) + m)
    result = least_squares(residuals, initial_guess)
    optimized_params = result.x
    A_opt = optimized_params[:m*m].reshape(m, m)
    b_opt = optimized_params[m*m:]
    
    return A_opt, b_opt

A_opt, b_opt = affine_transformation(X, Y)

print("Optimized A:")
print(A_opt)
print("Optimized b:")
print(b_opt)

Optimized A:
[[-3.05576183e-01  3.66827517e-01  1.82466468e+00  9.31846962e-01
   6.25945053e-01  1.86865320e+00  8.09803612e-01 -2.64995032e+00
  -1.95681268e+01  5.15337527e+00]
 [-5.93160470e-01 -3.12347886e-01 -6.61773886e-01 -4.74806028e-01
  -7.11568308e-01 -4.58321494e-01 -4.95456147e-01  6.40778314e-01
   3.66814494e+00 -4.91705486e-01]
 [-5.58187050e-01 -8.89039976e-02  5.47680610e-01  3.04891419e-02
  -1.21371152e+00 -1.54019834e+00 -5.25199514e-03 -5.18059545e-01
  -5.40668024e+00  1.43527657e+00]
 [-1.59684317e-01 -2.93652407e-01  2.37214494e-01  5.61081503e-01
   3.77990220e-01  1.18700975e+00  1.75370752e-01 -3.98110132e-01
  -3.19989236e+00  2.20245792e+00]
 [ 2.08582404e-01 -1.02604093e-02 -2.08026831e-01 -1.54688494e-01
   4.67150903e-01  2.15167647e-01 -6.80840357e-02  4.90616888e-01
   2.90582486e+00 -6.91321021e-01]
 [-8.51197076e-02 -1.14830530e-02  2.87683604e-02  9.45849087e-02
   3.13454447e-01  4.47960065e-01  3.55133570e-02 -2.77218538e-01
  -1.05475423e+00  3